In [1]:
from time import time
import configparser
import matplotlib.pyplot as plt
import pandas as pd
import boto3
import json
import sql_queries
import psycopg2
import create_tables
import etl

### I read the config file again to access and interact with the Redshift Cluster.

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('project_dwh.cfg'))

DWH_IAM_ROLE_NAME = config.get('IAM_ROLE','DWH_IAM_ROLE_NAME')

DB_NAME = config.get('CLUSTER','DB_NAME')
HOST = config.get('CLUSTER','HOST')
DB_USER = config.get('CLUSTER','DB_USER')
DB_PASSWORD = config.get('CLUSTER','DB_PASSWORD')
DB_PORT = config.get('CLUSTER','DB_PORT')
DWH_ENDPOINT = config.get('CLUSTER','DWH_ENDPOINT')

In [ ]:
create_tables.main()

### I initiated the connection to the Redshift Cluster

In [ ]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(DWH_ENDPOINT,DB_NAME,DB_USER,DB_PASSWORD,DB_PORT))
cur = conn.cursor()


### Here I drop and recreate the necessary tables: the 2 staging tables and other star schema tables

In [ ]:
create_tables.drop_tables(cur,conn)
create_tables.create_tables(cur,conn)

In [ ]:
#conn.close()

### Here I load the 2 staging tables with the data from S3 buckets.
#### To save time, I only used the files in the A/A/ folder of the song_data bucket.

In [ ]:
etl.load_staging_tables(cur,conn)

### The data in the staging tables are loaded to the production tables inside the Redshift Cluster.
#### Since, this is a SQL-to-SQL ETl, I used the combination of Insert Into and Select statements. We see which queries are ran without any error messages.

In [ ]:
etl.insert_tables(cur,conn)